# 号称最好的中文分词Python模块
P.S. : 以下的比较内容参考自jieba库作者的github的主页使用教程  
## 算法
* 前缀树
* 动态规划
* 采用了基于汉字成词能力的 HMM 模型，使用了 Viterbi 算法
## 功能
1.分词  
    支持三种分词模式：
        * 精确模式，试图将句子最精确地切开，适合文本分析；
        * 全模式，把句子中所有的可以成词的词语都扫描出来, 速度非常快，但是不能解决歧义；
        * 搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。
    带分词的字符串可以是 UTF-8 , Unicode(意味着python3内置的编码模式可以直接使用) , GBK(极度不推荐)
    1. 函数
        * jieba.cut(string , cut_all , HMM) : string是要分词的字符串，cut_all是选择是否采用全模式，HMM决定是否采用HMM模型,返回迭代器
        * jieba.cut_for_search（string , HMM） : string是要分词的字符串，HMM参数决定是否采用HMM模型，分词的粒度比较细，返回迭代器
        * jieba.lcut : 同上返回列表
        * jieba.cut_for_search : 同上返回列表
        * jieba.Tokenizer(dictionary=DEFAULT_DICT) : 新建自定义分词器，可用于同时使用不同词典。
        * jieba.dt : jieba.dt 为默认分词器，所有全局分词相关函数都是该分词器的映射。

In [1]:
import jieba

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.167 seconds.
Prefix dict has been built succesfully.


Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造


2.自定义词典  
* 建议为jieba载入jieba没有识别的词语，可以用户载入自定义字典
* 函数 :
    * jieba.load_userdict(file_name) : 载入用户自定义的字典文件，词典是是有格式的  
      词典格式和 dict.txt 一样，一个词占一行；每一行分三部分：词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒。file_name 若为路径或二进制方式打开的文件，则文件必须为 UTF-8 编码。**词频省略的时候可以自动计算**
    * add_word(word, freq, tag)  
        添加单词和词频，还有词性
    * del_word(word)   
        删除词典中的词
    * suggest_freq(segment, tune=True)  
        修改单个单词的次词频，保证不会被切分(或者切分)

In [3]:
import jieba
print('/'.join(jieba.cut('如果放到post中将出错。', HMM=False)))
jieba.suggest_freq(('中', '将'), True)
print('/'.join(jieba.cut('如果放到post中将出错。', HMM=False)))
print('/'.join(jieba.cut('「台中」正确应该不会被切开', HMM=False)))
# jieba.suggest_freq('台中', True)
# jieba.add_word('台中', 0)    # 没用，如果词频太小也没用
jieba.add_word('台中', freq=200)    # 正确的分出来了
print('/'.join(jieba.cut('「台中」正确应该不会被切开', HMM=False)))

如果/放到/post/中/将/出错/。
如果/放到/post/中/将/出错/。
「/台/中/」/正确/应该/不会/被/切开
「/台中/」/正确/应该/不会/被/切开


3.关键词提取  
* TF-IDF 算法  
import jieba.analyse

    * jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())
        * sentence 为待提取的文本
        * topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20
        * withWeight 为是否一并返回关键词权重值，默认值为 False
        * allowPOS 仅包括指定词性的词，默认值为空，即不筛选
    * jieba.analyse.TFIDF(idf_path=None) 新建 TFIDF 实例，idf_path 为 IDF 频率文件
* TextRank 算法

    * jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')) 直接使用，接口相同，注意默认过滤词性。
    * jieba.analyse.TextRank() 新建自定义 TextRank 实例


In [3]:
import jieba.analyse as ja
print(ja.extract_tags('舍友正在看直播'))
print(ja.textrank("舍友正在看直播"))
print(ja.extract_tags('我真的什么都不知道'))

['舍友', '直播', '正在']
['舍友', '直播']
['真的', '知道', '什么']


4.词性标注  
采用和ictclas兼容的标注方式
   * n : 名词
   * v : 动词
   * a : 形容词
   * b : 区别词
   * z : 状态词
   * r : 代词
   * m : 数词
   * q : 量词
   * d : 副词
   * p : 介词
   * c : 连词
   * u : 助词
   * e : 叹词
   * y : 语气词
   * o : 拟声词
   * h : 前缀
   * k : 后缀
   * w : 标点符号

In [15]:
import jieba.posseg as pseg
words = pseg.cut("我爱北京天安门")
for word, flag in words:
    print('%s %s' % (word, flag))

我 r
爱 v
北京 ns
天安门 ns
